<a href="https://colab.research.google.com/github/Pradeep23-01/Keras-tuner-experimental/blob/main/Keras_tuner_without_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=dd5cd599befc4df5dc5b6868e3635e383aa0d14417d9a30450e79f940e9baafb
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=1b95f17542461f21287a0d69bb728c6c5b2fde10d0e83885a27deeac3b25b26f
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
#Saving the dataset locally from keras
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_img,train_label),(test_img,test_label)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
#Normalizing the pixels to values b/w 0 and 1 
train_img=train_img/255.0
test_img=test_img/255.0

In [ ]:
train_img[0].shape

(28, 28)

In [ ]:
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

In [ ]:
from keras.layers import Conv2D,Flatten,Dense

In [ ]:
def build_model(hp):
  model=keras.models.Sequential([
                   Conv2D( filters=hp.Int('conv_filter1', min_value=64, max_value=128, step=16),
                           kernel_size=hp.Choice('conv_kernel1', values=[3,5]), 
                           activation='relu',
                           input_shape=(28,28,1) ),
                   
                   Conv2D( filters=hp.Int('conv_filter2', min_value=32, max_value=64, step=16),
                           kernel_size=hp.Choice('conv_kernel2', values=[3,5]), 
                           activation='relu'),
                   
                   Flatten(),
                   Dense(units=hp.Int('dense_units',min_value=32, max_value=128, step=16), 
                         activation='relu'),
                   Dense(units=10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])), 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,objective='val_accuracy', max_trials=5, directory='output', project_name='Mnist Fashion')

In [ ]:
tuner_search.search(train_img,train_label,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.8726666569709778

Best val_accuracy So Far: 0.9151666760444641
Total elapsed time: 00h 02m 29s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        102464    
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2973792   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,077,866
Trainable params: 3,077,866
Non-trainable params: 0
_________________________________________________________________
